# Libraries

In [142]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
#from sklearn.externals import joblib

# Configs 

In [170]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)

# Functions

In [144]:
def transform_dataset_train(df,inputString, numberMonths):
    df = df.sort_values(['ncodpers','fecha_dato'])
    df['rowNB']  = df.groupby(['ncodpers'])['fecha_dato'].cumcount()+1
    df_aux = df.copy()
    df_aux.columns = [str(col) + f"{inputString}"  for col in df_aux.columns]
    df2 = df.merge(df_aux, left_on=['ncodpers'], right_on=['ncodpers' + f"{inputString}"],how='left')    
    df2['rowDif'] = df2['rowNB' + f"{inputString}"]- df2.rowNB
    df3 = df2[(df2.ncodpers == (df2['ncodpers' + f"{inputString}"])) & (df2.rowDif==numberMonths)]
    return df3

# Data Importation 

In [145]:
train = pd.read_csv(r'C:\Users\Eugenio\Desktop\Kaggle\train_new.csv')

C:\Users\Eugenio\anaconda3\envs\env_eug\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5,8,10,11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [146]:
df = train.sample(100000)

In [224]:
df.columns = df.columns.str.replace(' ', '', regex=True)

In [221]:
df['age']        = df['age'].replace({np.nan:0, 'NA':0})
df['antiguedad'] = df['antiguedad'].replace({np.nan:0, 'NA':0})
df['renta']      = df['renta'].replace({np.nan:0, 'NA':0})

In [241]:
int_columns = ['age', 'antiguedad', 'renta']
    
#df[int_columns] = df[int_columns].astype(np.float64)

### Create historical dataset

#### add 1 month before to train

In [149]:
df2 = transform_dataset_train(df,'_1_after',1)

In [150]:
df2['fecha_dato_1_after'] = pd.to_datetime(df2['fecha_dato_1_after'], format='%Y-%m-%d')

### split train test

In [151]:
df2.columns[df2.isnull().any()]
df2.fillna(0,inplace=True)

In [152]:
df2.columns[df2.isnull().any()]

Index([], dtype='object')

In [153]:
df2.drop(['rowDif','rowNB_1_after','rowNB'], axis=1, inplace=True)

In [154]:
max_date     = df2['fecha_dato_1_after'].max()



cat_features = ['ind_empleado', 'pais_residencia', 'sexo', 
                'ult_fec_cli_1t', 'indrel_1mes', 'tiprel_1mes', 'indresi', 'indext', 
                'conyuemp', 'canal_entrada', 'indfall', 'nomprov', 'segmento', 
                'ind_empleado_1_after', 'pais_residencia_1_after', 'sexo_1_after',
                 'ult_fec_cli_1t_1_after', 'indrel_1mes_1_after', 'tiprel_1mes_1_after', 
                'indresi_1_after', 'indext_1_after', 
                'conyuemp_1_after', 'canal_entrada_1_after', 'indfall_1_after', 'nomprov_1_after', 'segmento_1_after']

In [155]:
df3 = pd.get_dummies(df2, columns=cat_features)

In [163]:
df3.columns[df3.isnull().any()]

Index([], dtype='object')

In [197]:
df3[df3.apply(lambda row: row.astype(str).str.contains('NA').any(), axis=1)].columns

,fecha_dato,ncodpers,age,fecha_alta,ind_nuevo,antiguedad,indrel,tipodom,cod_prov,ind_actividad_cliente,renta,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1,fecha_dato_1_after,ncodpers_1_after,age_1_after,fecha_alta_1_after,ind_nuevo_1_after,antiguedad_1_after,indrel_1_after,tipodom_1_after,cod_prov_1_after,ind_actividad_cliente_1_after,renta_1_after,ind_ahor_fin_ult1_1_after,ind_aval_fin_ult1_1_after,ind_cco_fin_ult1_1_after,ind_cder_fin_ult1_1_after,ind_cno_fin_ult1_1_after,ind_ctju_fin_ult1_1_after,ind_ctma_fin_ult1_1_after,ind_ctop_fin_ult1_1_after,ind_ctpp_fin_ult1_1_after,ind_deco_fin_ult1_1_after,ind_deme_fin_ult1_1_after,ind_dela_fin_ult1_1_after,ind_ecue_fin_ult1_1_after,ind_fond_fin_ult1_1_after,ind_hip_fin_ult1_1_after,ind_plan_fin_ult1_1_after,ind_pres_fin_ult1_1_after,ind_reca_fin_ult1_1_after,ind_tjcr_fin_ult1_1_after,ind_valo_fin_ult1_1_after,ind_viv_fin_ult1_1_after,ind_nomina_ult1_1_after,ind_nom_pens_ult1_1_after,ind_recibo_ult1_1_after,ind_empleado_0,ind_empleado_A,ind_empleado_B,ind_empleado_F,ind_empleado_N,pais_residencia_0,pais_residencia_AD,pais_residencia_AR,pais_residencia_AT,pais_residencia_AU,pais_residencia_BE,pais_residencia_BG,pais_residencia_BR,pais_residencia_BY,pais_residencia_CH,pais_residencia_CL,pais_residencia_CO,pais_residencia_CU,pais_residencia_CZ,pais_residencia_DE,pais_residencia_DO,pais_residencia_EC,pais_residencia_EG,pais_residencia_ES,pais_residencia_FR,pais_residencia_GB,pais_residencia_GN,pais_residencia_HK,pais_residencia_IL,pais_residencia_IN,pais_residencia_IT,pais_residencia_MA,pais_residencia_MX,pais_residencia_NG,pais_residencia_NL,pais_residencia_PL,pais_residencia_PT,pais_residencia_PY,pais_residencia_RO,pais_residencia_RU,pais_residencia_SA,pais_residencia_SE,pais_residencia_TR,pais_residencia_US,pais_residencia_VE,sexo_0,sexo_H,sexo_V,ult_fec_cli_1t_0,ult_fec_cli_1t_2015-10-06,ult_fec_cli_1t_2015-10-07,indrel_1mes_0,indrel_1mes_1.0,indrel_1mes_2.0,indrel_1mes_3.0,indrel_1mes_1,indrel_1mes_1.0,indrel_1mes_2.0,indrel_1mes_3.0,indrel_1mes_P,tiprel_1mes_0,tiprel_1mes_A,tiprel_1mes_I,tiprel_1mes_P,tiprel_1mes_R,indresi_0,indresi_N,indresi_S,indext_0,indext_N,indext_S,conyuemp_0,conyuemp_N,canal_entrada_0,canal_entrada_007,canal_entrada_013,canal_entrada_KAA,canal_entrada_KAB,canal_entrada_KAC,canal_entrada_KAD,canal_entrada_KAE,canal_entrada_KAF,canal_entrada_KAG,canal_entrada_KAH,canal_entrada_KAI,canal_entrada_KAJ,canal_entrada_KAK,canal_entrada_KAL,canal_entrada_KAM,canal_entrada_KAN,canal_entrada_KAO,canal_entrada_KAP,canal_entrada_KAQ,canal_entrada_KAR,canal_entrada_KAS,canal_entrada_KAT,canal_entrada_KAV,canal_entrada_KAW,canal_entrada_KAY,canal_entrada_KAZ,canal_entrada_KBB,canal_entrada_KBF,canal_entrada_KBG,canal_entrada_KBH,canal_entrada_KBJ,canal_entrada_KBO,canal_entrada_KBQ,canal_entrada_KBR,canal_entrada_KBS,canal_entrada_KBU,canal_entrada_KBV,canal_entrada_KBY,canal_entrada_KBZ,canal_entrada_KCA,canal_entrada_KCB,canal_entrada_KCC,canal_entrada_KCD,canal_entrada_KCF,canal_entrada_KCG,canal_entrada_KCH,canal_entrada_KCI,canal_entrada_KCL,canal_entrada_KCM,canal_entrada_KCN,canal_entrada_KCP,canal_entrada_KCU,canal_entrada_KDC,canal_entrada_KDD,canal_entrada_KDE,canal_entrada_KDF,canal_entrada_KDM,canal_entrada_KDO,canal_entrada_KDP,canal_entrada_KDR,canal_entrada_KDS,canal_entrada_KDU,canal_entrada_KDX,canal_entrada_KDY,canal_entrada_KDZ,canal_entrada_KEC,canal_entrada_KED,canal_entrada_KEG,canal_entrada_KEH,canal_entrada_KEI,canal_entrada_KEJ,canal_entrada_KEL,canal_entrada_KEN,canal_entrada_KEO,canal_entrada_KES,canal_entrada_KEV,canal_entrada_KEW,canal_entrada_KEY,canal_entrada_KEZ,canal_

In [208]:
import sys
np.set_printoptions(threshold=sys.maxsize)
df3.age.values

array([57, 50, 51, 51, 63, ' 46', ' 46', ' 46', ' 55', 62, 62, 44, ' 58',
       ' 58', 60, 52, ' 66', 53, ' 53', 63, 68, ' 53', 62, ' 86', ' 50',
       ' 49', 49, 49, ' 94', 94, 62, 71, 71, 72, 72, ' 74', 74, 59, ' 58',
       ' 69', ' 69', ' 35', ' 35', 47, ' 56', ' 68', 54, ' 70', 53, 53,
       66, ' 63', 60, ' 60', ' 58', ' 57', 46, ' 53', 54, 57, 91, ' 47',
       56, ' 55', ' 61', ' 73', 73, 73, 60, ' 56', ' 56', ' 56', ' 90',
       ' 82', 83, ' 56', ' 77', ' 72', 73, ' 48', 50, 51, ' 61', ' 48',
       ' 50', 62, 62, ' 49', ' 46', 63, ' 48', 48, 49, ' 60', 49, ' 54',
       ' 54', 50, 69, 51, 77, 55, 47, ' 56', 56, ' 62', 62, 62, 62, ' 54',
       ' 49', 57, 49, 49, ' 44', ' 52', 52, ' 56', ' 56', 55, ' 59', 58,
       ' 59', 60, ' 75', 61, 49, ' 78', ' 49', ' 60', 52, 52, ' 47',
       ' 48', 48, 72, 57, ' 55', ' 48', ' 64', ' 59', 59, ' 46', ' 77',
       71, 58, ' 89', ' 89', ' 81', ' 81', ' 81', 56, 69, ' 50', 50,
       ' 67', ' 60', ' 69', ' 73', 73, 47, 48, 57, 47, 80,

In [202]:
df3.age.fillna(0).value_counts()

23     1099
24      924
22      856
21      697
 22     647
 21     644
 23     586
25      576
 24     567
20      562
44      407
26      402
42      368
 25     367
27      356
 46     326
45      325
 44     317
43      316
46      316
 42     315
40      310
41      309
47      305
39      293
49      288
 45     286
 40     286
 43     279
 41     271
28      265
48      265
50      262
 47     260
 50     258
 48     255
 26     255
 49     247
52      246
 39     241
30      236
37      235
36      230
 27     227
53      224
 51     222
 38     220
51      220
38      219
29      215
35      209
 53     196
34      192
 52     180
 20     179
33      178
32      178
 37     176
 35     174
 28     173
57      168
54      165
55      165
31      164
 54     164
 36     161
 57     159
 30     158
 55     148
 29     145
 56     145
 32     145
56      145
58      143
60      141
 58     138
59      135
 59     133
 34     130
 33     125
 31     124
 60     122
61      117
62  

In [201]:
df3['age'] = df3['age'].replace({np.nan:0, 'NA':0})

In [188]:
test  = df3.loc[df3.fecha_dato_1_after == max_date]
train = df3.loc[df3.fecha_dato_1_after != max_date]

In [189]:
test.drop(['fecha_dato', 'ncodpers','fecha_alta','ncodpers_1_after','fecha_dato_1_after', 'fecha_alta_1_after'], axis=1, inplace=True)
train.drop(['fecha_dato', 'ncodpers','fecha_alta','ncodpers_1_after','fecha_dato_1_after', 'fecha_alta_1_after'], axis=1, inplace=True)

C:\Users\Eugenio\anaconda3\envs\env_eug\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [190]:
print(test.shape)
print(train.shape)

(3428, 537)
(23842, 537)


In [191]:
y_list = ['ind_ahor_fin_ult1_1_after', 'ind_aval_fin_ult1_1_after', 
'ind_cco_fin_ult1_1_after', 'ind_cder_fin_ult1_1_after', 'ind_cno_fin_ult1_1_after', 'ind_ctju_fin_ult1_1_after', 
'ind_ctma_fin_ult1_1_after', 'ind_ctop_fin_ult1_1_after', 'ind_ctpp_fin_ult1_1_after', 'ind_deco_fin_ult1_1_after', 
'ind_deme_fin_ult1_1_after', 'ind_dela_fin_ult1_1_after', 'ind_ecue_fin_ult1_1_after', 'ind_fond_fin_ult1_1_after', 
'ind_hip_fin_ult1_1_after', 'ind_plan_fin_ult1_1_after', 'ind_pres_fin_ult1_1_after', 'ind_reca_fin_ult1_1_after', 
'ind_tjcr_fin_ult1_1_after', 'ind_valo_fin_ult1_1_after', 
'ind_viv_fin_ult1_1_after', 'ind_nomina_ult1_1_after', 'ind_nom_pens_ult1_1_after', 'ind_recibo_ult1_1_after']

In [192]:
X_train = train.loc[:,~train.columns.isin(y_list)]
X_test  = test.loc[:,~test.columns.isin(y_list)]
y_train = train[y_list]
y_test  = test[y_list]

In [193]:
print(X_train.shape)
print(X_test.shape)
print(y_test.shape)
print(y_train.shape)

(23842, 513)
(3428, 513)
(3428, 24)
(23842, 24)


# Model

In [237]:
from sklearn.multioutput import MultiOutputClassifier


clf = MultiOutputClassifier(RandomForestClassifier())
clf.fit(X_train.drop(['age', 'antiguedad','age_1_after','antiguedad_1_after'], axis=1), y_train)

MultiOutputClassifier(estimator=RandomForestClassifier())

In [259]:
# Predicting the Test set results
y_pred = clf.predict(X_test.drop(['age', 'antiguedad','age_1_after','antiguedad_1_after'], axis=1))


In [260]:
y_pred

array([[0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,
        0., 1., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 1., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 1., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
        0., 1., 1., 0., 0., 1., 1., 1.],
       [0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
        0., 1., 0., 0., 

In [240]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix



print(f1_score(y_test, y_pred, average="macro"))
print(precision_score(y_test, y_pred, average="macro"))
print(recall_score(y_test, y_pred, average="macro"))   

0.7182236787555691
0.7808295153090565
0.6768003439858438


C:\Users\Eugenio\anaconda3\envs\env_eug\lib\site-packages\sklearn\metrics\_classification.py:1492: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
C:\Users\Eugenio\anaconda3\envs\env_eug\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Eugenio\anaconda3\envs\env_eug\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
